# Testing the solution


## Example


In [1]:
TEXT_LOADED = "Marinalva Bete Raz e Jorge Luiz receberam R$ 3.829,83 reais."
DADOS_LOADED = {
    "NOME": ["Marinalva Bete Raz", "Jorge Luiz"],
    "DINHEIRO": ["R$ 3.829,83"],
}

In [16]:
EXAMPLES = [
    [
        """EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM CRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA, O autor, Daniel Mendes, nascido em dezembro de 1990, com CPF 490.183.567-10 e telefone 0800 170 6459, é proprietário do endereço Praia Antônio Caldeira, 4, no CEP 28866-051. Ele também é dono da conta bancária com saldo de 43,95 e pode ser contatado pelo e-mail santosbarbara@example.net. O Ministério Público, representado pelo CPF 127.034.685-81 e telefone 61 6556 4995, e-mail frezende@example.net, também pode ser contatado. O valor da indenização é de RS 3,58 e o autor tem um prazo para depositar o dinheiro no banco."""
    ],
    [
        """Falsificação de documento público, uso de documento falso. Anotação de Responsabilidade Técnica Nº 1055330634101 de 12 de janeiro de 2013 relativa à Lei Federal Nº 531. 
 
Trata-se de representação referente a possível falsificação documentação técnica registrada pelo CREA-SP, feita pelo senhor Francis Pantele da Cozzi, CPF: 412.612.341-32, telefone (31) 951358433, email fran@bol.com, atinente à sua contratação pela senhora Marinalva Bete Raz, CPF: 049.567.041-22, telefone (61) 9412 3333, mulher branca, opinião política conservadora, religião evangélica. 

Marinalva Bete Raz reclama por indenização por danos morais no dia 14.05.2013 no valor de R$ 82.662,00 (Oitenta e dois mil, seiscentos e sessenta e dois reais) relacionado ao endereço IP 192.168.01 e ao endereço constante no CEP 59123-222, Rua dos Pioneiros, nº 450, Jardim Esmeralda, Campo Grande, MS. 

Ela afirma que precisa deste dinheiro o mais rápido possível, já que sua saúde está comprometida, pois sofre de asma. 

 A parte Francis Pantele da Cozzi afirma que apenas possui 5.534,00 reais como patrimônio e precisa do dinheiro para sustentar seu filho menor de idade chamado Josué Vittas. """
    ],
]

## Loading the model


In [2]:
from transformers import pipeline, AutoTokenizer
import torch


def get_transformer():
    model_name = "pierreguillou/ner-bert-large-cased-pt-lenerbr"
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        max_length=512,
        model_max_length=512,
        truncation=True,
    )
    return pipeline(
        "ner",
        tokenizer=tokenizer,
        model=model_name,
        aggregation_strategy="first",
        device=0 if torch.cuda.is_available() else -1,
    )


TRANSFORMER_MODEL = get_transformer()

/home/celiudos/miniconda3/envs/trc/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Loading the model and regex


In [13]:
from utils.ModeloNerUtils import (
    ModeloNerUtils,
)

ModeloNerUtilsObj = ModeloNerUtils(transformer=TRANSFORMER_MODEL)
resposta_gerada = ModeloNerUtilsObj.run(TEXT_LOADED)
resposta_gerada

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


('<NOME> e <NOME_2> receberam <DINHEIRO> reais.',
 {'NOME': {'<NOME>': 'Marinalva Bete Raz', '<NOME_2>': 'Jorge Luiz'},
  'DINHEIRO': {'<DINHEIRO>': 'R$ 3.829,83'}},
 {'text': '<NOME> e <NOME_2> receberam <DINHEIRO> reais.',
  'entities': [{'entity_group': 'NOME',
    'word': '<NOME>',
    'start': 0,
    'end': 6},
   {'entity_group': 'NOME', 'word': '<NOME_2>', 'start': 9, 'end': 17},
   {'entity_group': 'DINHEIRO',
    'word': '<DINHEIRO>',
    'start': 28,
    'end': 38}]})

## Loading UI


In [18]:
import gradio as gr


def show_ner(text):
    texto_anonimizado, segredo_de_anonimizacao, entidades_gradio = (
        ModeloNerUtilsObj.run(text)
    )
    return entidades_gradio, segredo_de_anonimizacao


gr.Interface(
    fn=show_ner,
    theme="soft",
    js="() => {document.body.classList.remove('dark')}",
    title="Evaluation",
    inputs=[
        gr.Textbox(label="Input", lines=3),
    ],
    outputs=[
        gr.HighlightedText(
            label="Output",
            show_legend=True,
        ),
        gr.JSON(
            label="Entities",
        ),
    ],
    live=True,
    examples=EXAMPLES,
    allow_flagging="never",
).launch(
    inbrowser=True,
)

Running on local URL:  http://127.0.0.1:7871


/home/celiudos/miniconda3/envs/trc/lib/python3.9/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(



To create a public link, set `share=True` in `launch()`.
